## Import datas

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Use pandas to loadinto a DataFrame
# Y1.csv doesn’t have a header so
# add one when loading the file
X1 = pd.read_csv("data/X1.csv")
Y1 = pd.read_csv("data/Y1.csv", header=None, names=['revenue'])


# ENLEVER colonne 'Unnamed: 0', 'img_url', 'description' du dataset
X = X1.drop(['Unnamed: 0', 'img_url', 'description'], axis=1)

# Creer un data training/validation splités a partir du X1 (on garde X2 pour les vrais tests) 
x_train, x_test, y_train, y_test = train_test_split(X, Y1, test_size = 0.2, random_state=1)
x_train[:10]
# lr = LinearRegression().fit(x_train,y_train)
# lr.score(x_test,y_test)

,title,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings
1688,Mini's First Time,5.8,8057.0,0,2006,91,"Comedy,Crime,Drama",2006.0,FInd.,"[0.75266373, 0.82829547, 1.0711715, 0.13163932...","[-0.6548688, 0.41124612, 0.999657, -0.98240334..."
3415,Matilda,6.9,131336.0,0,1996,98,"Comedy,Family,Fantasy",1996.0,Sony,"[0.20639575, 0.6939381, 0.6786564, 0.28500003,...","[-0.639669, 0.35792068, 0.9996606, -0.98308694..."
2847,Out for Justice,6.1,21662.0,0,1991,91,"Action,Crime,Drama",1991.0,WB,"[0.35394654, 2.1365995, 0.031171178, 0.588205,...","[-0.75107455, 0.445656, 0.9998593, -0.98727596..."
2748,Dog Lover's Symphony,2.9,115.0,0,2006,103,"Comedy,Family",2006.0,DogL,"[0.36566886, 1.4806824, 1.1436529, 0.13148876,...","[-0.7769195, 0.47293696, 0.99996483, -0.996776..."
2343,Eureka,7.9,48460.0,0,2006,60,"Adventure,Comedy,Drama",2001.0,Shoot.,"[0.16137362, 0.8321404, 0.706822, 0.05146308, ...","[-0.72176015, 0.42504603, 0.9998175, -0.983217..."
1185,Iceman,7.7,542.0,0,2007,43,"Action,Crime,Drama",1984.0,Uni.,"[0.024602404, 0.27386987, 0.6444133, 0.4109535...","[-0.43452877, 0.17764756, 0.9968394, -0.951669..."
2715,The Kiss,8.0,7.0,0,2005,\N,Documentary,1988.0,TriS,"[0.36038196, 1.2203285, 0.987185, 0.19289121, ...","[-0.73109615, 0.41096327, 0.99979013, -0.98975..."
3005,All Over Me,6.4,3143.0,0,1997,86,"Crime,Drama,Romance",1997.0,FL,"[0.10517039, 3.666064, 0.9746939, 0.36922383, ...","[-0.6584357, 0.4504648, 0.9997562, -0.98165154..."
3304,The Assassination of Richard Nixon,6.9,25190.0,0,2004,95,"Biography,Crime,Drama",2004.0,Think,"[0.33894196, 1.7841934, 0.581968, 0.05220865, ...","[-0.7256267, 0.3914477, 0.99981946, -0.9835853..."
3703,Terminal Velocity,5.5,12895.0,0,1994,102,"Action,Mystery,Romance",1994.0,BV,"[0.23503634, 2.4894485, 0.31714305, 0.35130754...","[-0.7101979, 0.41186222, 0.9997788, -0.9846159..."


### Compute the Root Mean Square Error

In [ ]:
def compute_rmse(predict, target):
    if len(target.shape) == 2:
        target = target.squeeze()
    if len(predict.shape) == 2:
        predict = predict.squeeze()
    diff = target - predict
    if len(diff.shape) == 1:
        diff = np.expand_dims(diff, axis=-1)
    rmse = np.sqrt(diff.T@diff / diff.shape[0])
    return float(rmse)